In [5]:
#importazione dei dati ------------------------------------------------------------------------------------------------------------

from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.linalg import toeplitz
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

df = pd.read_csv (r'/Users/patrickpinta/Desktop/1datasetpanda/corn.csv',header=0,error_bad_lines=False) 

#modellazione dei dati training set------------------------------------------------------------------------------------------------------------
xExpUsa= df.iloc[0:334,1:2]
xExpUsa = np.array(xExpUsa)
xExpEu=df.iloc[0:334,3:4]
xExpEu = np.array(xExpEu)
xImportChina=xExpUsa+xExpEu

xBimestri=df.iloc[0:334,13:18]
xBimestri = np.array(xBimestri)

xUsa=df.iloc[0:334,0:1]
xUsa = np.array(xUsa)
xBimInfExpUsa=xBimestri*xUsa

xEu=df.iloc[0:334,1:2]
xEu = np.array(xEu)
xBimInfExpEu=xBimestri*xEu

x= df.iloc[0:334,0:1]
x1= df.iloc[0:334,2:3]
x2= df.iloc[0:334,4:5]
x3= df.iloc[0:334,6:18]

xImportChina=xImportChina.tolist()
xImportChina = DataFrame (xImportChina,columns=['ImportChina'])
xBimInfExpUsa=xBimInfExpUsa.tolist()
xBimInfExpUsa = DataFrame (xBimInfExpUsa,columns=['bim1*expUsa','bim2*expUsa','bim3*expUsa','bim4*expUsa','bim5*expUsa'])
xBimInfExpEU=xBimInfExpEu.tolist()
xBimInfExpEu = DataFrame (xBimInfExpEu,columns=['bim1*expEu','bim2*expEu','bim3*expEu','bim4*expEu','bim5*expEu'])
x=x.join(xImportChina)
x=x.join(x1)
x=x.join(x2)
x=x.join(x3)
x=x.join(xBimInfExpUsa)
x=x.join(xBimInfExpEu)

y= df.iloc[0:334,19:20]

print(x)

#stardadizzazione dei dati training set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_train_stand=(scaler.fit_transform(x))
    
#modello dati training set------------------------------------------------------------------------------------------------------------    
    
x_train_stand, y = np.array(x_train_stand), np.array(y)
x_train_stand = sm.add_constant(x_train_stand)

#procedura per ols model ------------------------------------------------------------------------------------------------------------

ols_model = sm.OLS(y, x_train_stand)
ols_results = ols_model.fit(cov_type='HC1') 
print(ols_results.summary())

reg = linear_model.LinearRegression()
reg= reg.fit(x_train_stand,y)

#forecast ------------------------------------------------------------------------------------------------------------

ypredTrain = reg.predict(x_train_stand)

print('r2 train: ',r2_score(y,ypredTrain))
print('mse train: ',mean_squared_error(y,ypredTrain))

scores=cross_validate(reg, x_train_stand, y, cv=5,scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
print('r2 cv: ', scores['test_r2'].mean())
print('mse cv: ', abs(scores['test_neg_mean_squared_error'].mean()))








     export_usa  ImportChina  export_eu  openInterest  oneYearTreasuryRate  \
0        599.00         0.01     206.23       1528000                1.444   
1        690.82         2.49      61.39       1484000                1.665   
2        977.40       135.47      81.38       1199000                0.124   
3        889.27         2.74      30.05       1543000                1.803   
4        494.28         0.79     195.87       1609000                1.607   
..          ...          ...        ...           ...                  ...   
329     1027.85       421.43      74.28       1722000                0.112   
330      831.09         1.71     100.37       1301000                0.157   
331     1290.27         0.00     117.15       1389000                0.150   
332      386.94        27.52     125.85       1328000                0.145   
333      377.40         0.10      36.72       1178000                0.106   

     prec_usa  tempMaxUsa   prec_ue  tempAvg_ue  stockToUseWorl